In [4]:
import numpy as np
import pandas as pd
import joblib
from pathlib import Path

# =========================
# CONFIG
# =========================
DATA_ROOT = Path("/Users/jakobschneider/Machine Learning/Data_LCC")

MODEL_TABLE_PATH = DATA_ROOT / "model_table_imo_2024.parquet"
MRV_PATH         = DATA_ROOT / "MRV_2024.xlsx"
MODEL_PATH       = DATA_ROOT / "models" / "log_regression_mrv_validation.joblib"

OUT_PATH         = DATA_ROOT / "reporting" / "ship_report_imo_2024.parquet"
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)

# Must match trained pipeline
NUM_FEATURES = ["sog_p50_kn", "sog_p95_kn", "moving_share", "Length", "Width", "draft_m_median"]
CAT_FEATURES = ["VesselType"]

# MRV columns (as provided)
MRV_IMO_COL    = "IMO Number"
MRV_NAME_COL   = "Name"
MRV_TYPE_COL   = "Ship type"
MRV_PERIOD_COL = "Reporting Period"
MRV_TARGET_COL = "CO₂ emissions per distance [kg CO₂ / n mile]"

# =========================
# HELPERS
# =========================
def clean_imo(s: pd.Series) -> pd.Series:
    """
    Normalize IMO column from Excel/parquet to Int64.
    Handles floats like '1234567.0' and stray non-digit characters.
    """
    s2 = (s.astype(str)
            .str.strip()
            .str.replace(r"\.0$", "", regex=True)
            .str.replace(r"\D+", "", regex=True))
    return pd.to_numeric(s2, errors="coerce").astype("Int64")

# =========================
# LOAD
# =========================
df_ais = pd.read_parquet(MODEL_TABLE_PATH)
df_mrv = pd.read_excel(MRV_PATH)
pipe   = joblib.load(MODEL_PATH)

# =========================
# CLEAN KEYS
# =========================
df_ais["IMO"] = clean_imo(df_ais["IMO"])
df_mrv["IMO"] = clean_imo(df_mrv[MRV_IMO_COL])

df_ais = df_ais.dropna(subset=["IMO"]).copy()
df_mrv = df_mrv.dropna(subset=["IMO"]).copy()

df_ais["IMO"] = df_ais["IMO"].astype(int)
df_mrv["IMO"] = df_mrv["IMO"].astype(int)

# =========================
# SELECT + CLEAN MRV FIELDS
# =========================
df_mrv_small = df_mrv[["IMO", MRV_NAME_COL, MRV_TYPE_COL, MRV_PERIOD_COL, MRV_TARGET_COL]].copy()
df_mrv_small = df_mrv_small.rename(columns={
    MRV_NAME_COL:   "ship_name",
    MRV_TYPE_COL:   "mrv_ship_type",
    MRV_PERIOD_COL: "mrv_reporting_period",
    MRV_TARGET_COL: "y_mrv_co2_per_nm_kg",
})

# Make target numeric, then drop missing immediately
df_mrv_small["y_mrv_co2_per_nm_kg"] = pd.to_numeric(df_mrv_small["y_mrv_co2_per_nm_kg"], errors="coerce")
df_mrv_small = df_mrv_small.dropna(subset=["y_mrv_co2_per_nm_kg"]).copy()

# =========================
# JOIN (only IMOs present in both)
# =========================
df = df_ais.merge(df_mrv_small, on="IMO", how="inner")

# =========================
# DROP ALL MISSING VALUES IMMEDIATELY (your requirement)
# - ensure full feature completeness before prediction
# =========================
required_cols = ["IMO"] + NUM_FEATURES + CAT_FEATURES + ["y_mrv_co2_per_nm_kg"]
before = df.shape[0]
df = df.dropna(subset=required_cols).copy()
after = df.shape[0]

print("Rows before dropna:", before)
print("Rows after dropna :", after)
print("Dropped rows      :", before - after)

# Enforce correct dtypes
df["VesselType"] = df["VesselType"].astype("string")
for c in NUM_FEATURES + ["y_mrv_co2_per_nm_kg"]:
    df[c] = pd.to_numeric(df[c], errors="raise")

# =========================
# PREDICT (all remaining rows are complete)
# =========================
X = df[NUM_FEATURES + CAT_FEATURES]
df["y_pred_log"] = pipe.predict(X)
df["y_pred_co2_per_nm_kg"] = np.exp(df["y_pred_log"])

# =========================
# RESIDUALS
# =========================
df["residual_kg"] = df["y_mrv_co2_per_nm_kg"] - df["y_pred_co2_per_nm_kg"]
df["residual_pct"] = df["residual_kg"] / df["y_mrv_co2_per_nm_kg"]

# =========================
# GREEN/RED CLASSIFICATION
# =========================
abs_threshold = df["residual_kg"].abs().quantile(0.95)
rel_threshold = 0.30

df["flag_color"] = np.where(
    (df["residual_kg"].abs() > abs_threshold) | (df["residual_pct"].abs() > rel_threshold),
    "RED",
    "GREEN"
)

# transparent reason
df["flag_reason"] = "ok"
df.loc[df["residual_kg"].abs() > abs_threshold, "flag_reason"] = "abs_residual>p95"
df.loc[df["residual_pct"].abs() > rel_threshold, "flag_reason"] = "rel_residual>30%"

# =========================
# METADATA
# =========================
df["model_name"] = "log_regression_mrv_validation"
df["abs_threshold_residual_kg_p95"] = abs_threshold
df["rel_threshold_residual_pct"] = rel_threshold
df["report_year"] = 2024

# =========================
# SAVE
# =========================
df.to_parquet(OUT_PATH, index=False)

print("\nSaved:", OUT_PATH)
print("Rows:", df.shape[0], "| Unique IMOs:", df["IMO"].nunique())
print("\nFlag distribution:")
print(df["flag_color"].value_counts())
print("\nThreshold abs residual (p95):", abs_threshold)
print("Threshold rel residual:", rel_threshold)

Rows before dropna: 6490
Rows after dropna : 6210
Dropped rows      : 280

Saved: /Users/jakobschneider/Machine Learning/Data_LCC/reporting/ship_report_imo_2024.parquet
Rows: 6210 | Unique IMOs: 6210

Flag distribution:
flag_color
GREEN    5460
RED       750
Name: count, dtype: int64

Threshold abs residual (p95): 173.6794479643612
Threshold rel residual: 0.3


In [5]:
import pandas as pd

REPORT_PATH = "/Users/jakobschneider/Machine Learning/Data_LCC/reporting/ship_report_imo_2024.parquet"
df = pd.read_parquet(REPORT_PATH)

print("Shape:", df.shape)
print("Unique IMOs:", df["IMO"].nunique())

Shape: (6210, 32)
Unique IMOs: 6210


In [6]:
df.columns

Index(['IMO', 'ais_distance_nm_total', 'ais_time_hours_total', 'ais_points',
       'sog_mean_kn', 'sog_p50_kn', 'sog_p95_kn', 'moving_hours', 'idle_hours',
       'median_dt_seconds', 'VesselType', 'Length', 'Width', 'draft_m_median',
       'year', 'moving_share', 'quality_ok', 'y_co2_per_nm_kg', 'ship_name',
       'mrv_ship_type', 'mrv_reporting_period', 'y_mrv_co2_per_nm_kg',
       'y_pred_log', 'y_pred_co2_per_nm_kg', 'residual_kg', 'residual_pct',
       'flag_color', 'flag_reason', 'model_name',
       'abs_threshold_residual_kg_p95', 'rel_threshold_residual_pct',
       'report_year'],
      dtype='object')

In [17]:
ordered_cols = [
    # A) Stammdaten
    "IMO",
    "ship_name",
    "VesselType",
    "mrv_ship_type",
    "report_year",

    # B) AIS
    "ais_distance_nm_total",
    "ais_time_hours_total",
    "ais_points",
    "sog_mean_kn",
    "sog_p50_kn",
    "sog_p95_kn",
    "moving_share",
    "Length",
    "Width",
    "draft_m_median",

    # C) MRV
    "y_mrv_co2_per_nm_kg",

    # D) Regression
    "y_pred_co2_per_nm_kg",

    # E) Residuen & Flagging
    "residual_kg",
    "residual_pct",
    "flag_color",
    "flag_reason",
]

# keep only columns that actually exist (robust)
ordered_cols = [c for c in ordered_cols if c in df.columns]

df = df[ordered_cols].copy()
df.to_parquet(OUT_PATH, index=False)

In [14]:
import pandas as pd

REPORT_PATH = "/Users/jakobschneider/Machine Learning/Data_LCC/reporting/ship_report_imo_2024.parquet"
df = pd.read_parquet(REPORT_PATH)

print("Shape:", df.shape)
print("Unique IMOs:", df["IMO"].nunique())

Shape: (6210, 26)
Unique IMOs: 6210


In [15]:
df.columns

Index(['IMO', 'ship_name', 'VesselType', 'mrv_ship_type',
       'mrv_reporting_period', 'report_year', 'ais_distance_nm_total',
       'ais_time_hours_total', 'ais_points', 'sog_mean_kn', 'sog_p50_kn',
       'sog_p95_kn', 'moving_share', 'Length', 'Width', 'draft_m_median',
       'y_mrv_co2_per_nm_kg', 'y_pred_co2_per_nm_kg', 'y_pred_log',
       'model_name', 'residual_kg', 'residual_pct', 'flag_color',
       'flag_reason', 'abs_threshold_residual_kg_p95',
       'rel_threshold_residual_pct'],
      dtype='object')

In [16]:
df.head()

,IMO,ship_name,VesselType,mrv_ship_type,mrv_reporting_period,report_year,ais_distance_nm_total,ais_time_hours_total,ais_points,sog_mean_kn,...,y_mrv_co2_per_nm_kg,y_pred_co2_per_nm_kg,y_pred_log,model_name,residual_kg,residual_pct,flag_color,flag_reason,abs_threshold_residual_kg_p95,rel_threshold_residual_pct
0,1013676,AQUADONNA,70.0,Bulk carrier,2024,2024,374.749695,137.076667,2989,4.256307,...,234.32,263.288887,5.573252,log_regression_mrv_validation,-28.968887,-0.123630,GREEN,ok,173.679448,0.3
1,1014606,SEA GOAT,70.0,Bulk carrier,2024,2024,387.254333,30.897222,1138,12.567750,...,196.40,266.372617,5.584896,log_regression_mrv_validation,-69.972617,-0.356276,RED,rel_residual>30%,173.679448,0.3
2,1014618,BIRD OF PARADISE,70.0,Bulk carrier,2024,2024,1325.759888,435.353611,9706,4.851916,...,184.09,242.237417,5.489918,log_regression_mrv_validation,-58.147417,-0.315864,RED,rel_residual>30%,173.679448,0.3
3,1015313,CYTA,70.0,Bulk carrier,2024,2024,1676.166748,338.531944,9302,7.065696,...,206.75,239.153833,5.477107,log_regression_mrv_validation,-32.403833,-0.156730,GREEN,ok,173.679448,0.3
4,1015325,ANNELIESE,70.0,Bulk carrier,2024,2024,2395.103027,503.890000,11449,6.045777,...,201.22,247.281081,5.510526,log_regression_mrv_validation,-46.061081,-0.228909,GREEN,ok,173.679448,0.3
